In [2]:
#funcion que activa autocompletado de codigo para hacer mas facil la programacion
%config IPCompleter.greedy=True

In [3]:
#instalamos paquetes que no tenemos por defaul en la imagen de docker
import sys
!{sys.executable} -m pip install keras

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 377 kB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 45.9 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 11.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
#importamos librerias necesarias
import pandas as pd
import numpy as np
#from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
#from keras.models import Model
import random

Leemos el dataset que esta en el formato necesario pra cargarse en la libreria libmf 

In [8]:
df = pd.read_csv('/datos/datasets/netflix_complete_cleaned.txt',sep=' ', header = None, names = ['Cust_Id','Movie_Id','Rating'], usecols = [0,1,2])

Hacemos un resumen 

In [11]:
print('Dataset shape: {}'.format(df.shape))
print('Dataset types:')
print(df.dtypes)
print('Summary statistics from dataset:')
df.describe()

Dataset shape: (100480507, 3)
Dataset types:
Cust_Id       int64
Movie_Id      int64
Rating      float64
dtype: object
Summary statistics from dataset:


,Cust_Id,Movie_Id,Rating
count,1.004805e+08,1.004805e+08,1.004805e+08
mean,1.322489e+06,9.070915e+03,3.604290e+00
std,7.645368e+05,5.131891e+03,1.085219e+00
min,6.000000e+00,1.000000e+00,1.000000e+00
25%,6.611980e+05,4.677000e+03,3.000000e+00
50%,1.319012e+06,9.051000e+03,4.000000e+00
75%,1.984455e+06,1.363500e+04,4.000000e+00
max,2.649429e+06,1.777000e+04,5.000000e+00


Filtramos películas y usuarios con pocas evaluaciones para reducir el tamaño del dataset y mejorar predicciones. (Prune dataset)
Filtramos deacuerdo al percentil 60: 
Movie minimum times of review: 1006.0
Customer minimum times of review: 142.0

In [13]:
f = ['count','mean']

df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.6),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.6),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

Movie minimum times of review: 1006.0
Customer minimum times of review: 142.0


Se filtra el dataset original de acuerdo a los criterios de filtrado. 

In [14]:
# Filter sparse movies
min_movie_ratings = 1006
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 142
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings
print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

Shape User-Ratings unfiltered:	(100480507, 3)
Shape User-Ratings filtered:	(81822052, 3)


Hacemos un shuffle del dataframe y lo dividimos en entrenamiento y prueba

In [17]:
random.seed(28882)
#random shuffle dataset
df_filterd=df_filterd.sample(frac=1).reset_index(drop=True)
print(len(df_filterd))

#dividimos dataset en train y test
n=round(len(df_filterd)*.8) # Tamaño de muestra de entrenamiento (80% de los datos)
df_train=df_filterd[-n:]
print(len(df_train)) # Número de valores en el dataset de entrenamiento
df_test=df_filterd[:-n]
print(len(df_test)) # Número de valores en dataset de prueba


81822052
65457642
16364410


Imprimimos ejemplos del dataset de entrenamiento. 

In [19]:
df_train.to_csv(r'/datos/datasets/netflix_train.txt',header=False,sep = ' ',index=False)
print('Full dataset shape: {}'.format(df_train.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

Full dataset shape: (65457642, 3)
-Dataset examples-
           Cust_Id  Movie_Id  Rating
0          1488844         1     3.0
5000000     501954       996     2.0
10000000    404654      1962     5.0
15000000    886608      2876     2.0
20000000   1193835      3825     2.0
25000000   1899206      4661     3.0
30000000    154804      5496     4.0
35000000   2078749      6274     5.0
40000000    450763      7057     5.0
45000000    102092      7991     3.0
50000000    220298      9023     5.0
55000000    550530     10042     5.0
60000000    222570     11038     3.0
65000000   1273080     11875     5.0
70000000   2026970     12676     5.0
75000000    506044     13582     4.0
80000000    353605     14453     2.0
85000000    664606     15116     3.0
90000000   2213715     16008     3.0
95000000   1589401     16879     5.0
100000000  2314006     17627     4.0


Imprimimos ejemplos del dataset de prueba. 

In [20]:
df_test.to_csv(r'/datos/datasets/netflix_test.txt',header=False,sep = ' ',index=False)
print('Full dataset shape: {}'.format(df_test.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

Full dataset shape: (16364410, 3)
-Dataset examples-
           Cust_Id  Movie_Id  Rating
0          1488844         1     3.0
5000000     501954       996     2.0
10000000    404654      1962     5.0
15000000    886608      2876     2.0
20000000   1193835      3825     2.0
25000000   1899206      4661     3.0
30000000    154804      5496     4.0
35000000   2078749      6274     5.0
40000000    450763      7057     5.0
45000000    102092      7991     3.0
50000000    220298      9023     5.0
55000000    550530     10042     5.0
60000000    222570     11038     3.0
65000000   1273080     11875     5.0
70000000   2026970     12676     5.0
75000000    506044     13582     4.0
80000000    353605     14453     2.0
85000000    664606     15116     3.0
90000000   2213715     16008     3.0
95000000   1589401     16879     5.0
100000000  2314006     17627     4.0


Insertar en libreria libmf

In [21]:
!pwd

/datos/proyecto-final-equipo5-mno-2020-1/Implementation/Codigo_Exploratorio_LIBMF/Sampling_Design/Test2_Netflix_Total_Dataset
